In [1]:
import cv2
from skimage.metrics import structural_similarity
import pandas as pd
import numpy as np
import pickle
import psycopg2
import urllib.parse as urlparse
import os
## fast writes
from sqlalchemy import event, create_engine
import psycopg2.extras
import psycopg2.errorcodes

def open_connection():
    connection = psycopg2.connect(
        database=os.environ['RACK_DATABASE'],
        user=os.environ['RACK_USERNAME'],
        password=os.environ['RACK_PASSWORD'],
        host=os.environ['RACK_HOST'],
        port=os.environ['RACK_PORT']
        )
    return connection

import boto3
import os
s3 = boto3.resource('s3', region_name='us-east-1')
bucket = s3.Bucket('victims')

haar_cascade_face = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

```
CREATE TABLE match_results (
    input_bucket varchar(250),
    input_prefix varchar(250),
    input_filename varchar(250),
    similarity_score varchar(250),
    similarity_bucket varchar(250),
    similarity_prefix varchar(250),
    similarity_filename varchar(250)
);
```

In [9]:
df = pd.read_sql_query("select * from suspicious_activity_reports where id = 11;", open_connection())
df

,id,suspicion_type,date_observed,time_observed,room_number,room_registered_name,gender,age_observed,ethnicity,hair_color,...,license_number,license_state,why_vehicle_suspicious,anything_else,filename,bucket,prefix,url,race,location_observed
0,11,criminal,None,None,None,None,None,None,American Indian or Alaska Native,None,...,None,None,None,None,test.jpg,racksaruploads,11,https://racksaruploads.s3.amazonaws.com/11/tes...,None,None


In [10]:
df.loc[0]

id                                                                       11
suspicion_type                                                     criminal
date_observed                                                          None
time_observed                                                          None
room_number                                                            None
room_registered_name                                                   None
gender                                                                 None
age_observed                                                           None
ethnicity                                  American Indian or Alaska Native
hair_color                                                             None
hair_length                                                            None
noteable_features                                                      None
additional_victim                                                      None
additional_c

In [11]:
df_sub = df[['bucket', 'prefix', 'filename']].copy()
df_sub.columns = ['input_bucket', 'input_prefix', 'input_filename']
df_sub['similarity_score'] = 0.9133
df_sub['similarity_bucket'] = 'perpetrators'
df_sub['similarity_prefix'] = 1
df_sub['similarity_filename'] = 'j-epstein-match.jpg'

df_sub

,input_bucket,input_prefix,input_filename,similarity_score,similarity_bucket,similarity_prefix,similarity_filename
0,racksaruploads,11,test.jpg,0.9133,perpetrators,1,j-epstein-match.jpg


In [ ]:
df_sub

In [8]:
engine = create_engine('postgresql://{}:{}@{}/{}'.format(os.environ['RACK_USERNAME'],
                                                            os.environ['RACK_PASSWORD'],
                                                            os.environ['RACK_HOST'],
                                                            os.environ['RACK_DATABASE']
                                                           ))

In [12]:
df_sub.to_sql('match_results', con=engine, if_exists='append', index=False)

In [13]:
pd.read_sql_query('select * from match_results', open_connection())

,input_bucket,input_prefix,input_filename,similarity_score,similarity_bucket,similarity_prefix,similarity_filename
0,racksaruploads,11,test.jpg,0.9133,perpetrators,1,j-epstein-match.jpg


In [19]:
df = pd.read_sql_query("select * from suspicious_activity_reports where id = 17;", open_connection())
df

,id,suspicion_type,date_observed,time_observed,room_number,room_registered_name,gender,age_observed,ethnicity,hair_color,...,license_number,license_state,why_vehicle_suspicious,anything_else,filename,bucket,prefix,url,race,location_observed
0,17,victim,yesterday,2pm,None,None,male,None,American Indian or Alaska Native,None,...,None,None,None,None,vallow.jpg,racksaruploads,17,https://racksaruploads.s3.amazonaws.com/17/val...,White,lobby


In [20]:
df_sub = df[['bucket', 'prefix', 'filename']].copy()
df_sub.columns = ['input_bucket', 'input_prefix', 'input_filename']

df_sub['similarity_score'] = 0.884819168401895
df_sub['similarity_bucket'] = 'victims'
df_sub['similarity_prefix'] = 38
df_sub['similarity_filename'] = 'vallow.jpg'

df_sub

,input_bucket,input_prefix,input_filename,similarity_score,similarity_bucket,similarity_prefix,similarity_filename
0,racksaruploads,17,vallow.jpg,0.884819,victims,38,vallow.jpg


In [21]:
df_sub.to_sql('match_results', con=engine, if_exists='append', index=False)

In [22]:
pd.read_sql_query('select * from match_results', open_connection())

,input_bucket,input_prefix,input_filename,similarity_score,similarity_bucket,similarity_prefix,similarity_filename
0,racksaruploads,11,test.jpg,0.9133,perpetrators,1,j-epstein-match.jpg
1,racksaruploads,17,vallow.jpg,0.884819168401895,victims,38,vallow.jpg


In [2]:
haar_cascade_face = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
def convertToRGB(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

def get_gray(faces, image, gray):
    image_gray = None
    for (x,y,w,h) in faces:
        cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)
        image_gray = gray[y:y+h, x:x+w]
    return image_gray

def get_single_face(gray):
    for i in range(10, 25):
        faces = haar_cascade_face.detectMultiScale(gray, 1.1, i)
        if len(faces) == 1:
            return faces, True
        elif len(faces) == 0:
            return [], False
    return [], False

def get_gray(faces, image, gray):
    image_gray = None
    for (x,y,w,h) in faces:
        cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)
        image_gray = gray[y:y+h, x:x+w]
    return image_gray

def compare_images(hotel_image_path, directory, id_, file, return_image=False):
    ## format image to compare
    s3 = boto3.resource('s3', region_name='us-east-1')
    bucket = s3.Bucket('racksaruploads')
    bucket.download_file(hotel_image_path, 'tmp.jpg')
    image = cv2.imread('tmp.jpg')
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = haar_cascade_face.detectMultiScale(gray, 1.1, 2)
    gray_cap = get_gray(faces, image, gray)
    H, W = gray_cap.shape
    gray_cap_area = H*W    
        
        
    ## get victim/perp image
    s3 = boto3.resource('s3', region_name='us-east-1')
    bucket = s3.Bucket(directory)
    bucket.download_file('{}/{}'.format(id_, file), 'tmp.jpg')
    image = cv2.imread('tmp.jpg')
    
    ## format image
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces, found = get_single_face(gray)
    if found:
        gray_v = get_gray(faces, image, gray)

        ## prep for comparison
        H, W = gray_v.shape
        image_area = H*W
        

        if gray_cap_area > image_area:
            gray_v = cv2.resize(gray_v, gray_cap.shape)
        elif gray_cap_area < image_area:
            gray_cap = cv2.resize(gray_cap, gray_v.shape)
        
        if return_image == False:
            return structural_similarity(gray_cap, gray_v)
        else:
            return structural_similarity(gray_cap, gray_v), image
    if return_image == False:
        return None
    else:
        return None, None

In [3]:
compare_images(hotel_image_path='11/test.jpg', directory='victims', id_='36', file='person-wilsonzyaiaslg.jpg')

0.3363522938109242

Update missing
==

In [5]:
df_suspicious = pd.read_sql_query("select * from suspicious_activity_reports;", open_connection())
df_suspicious

,id,suspicion_type,date_observed,time_observed,room_number,room_registered_name,gender,age_observed,ethnicity,hair_color,...,license_number,license_state,why_vehicle_suspicious,anything_else,filename,bucket,prefix,url,race,location_observed
0,2,victim,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,3,victim,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,4,victim,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,5,victim,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,6,victim,2020-03-29,None,10,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
5,1,victim,None,None,None,None,None,None,None,None,...,None,None,None,None,https://racksaruploads.s3.amazonaws.com/1/Scre...,None,None,None,None,None
6,7,victim,2010-01-21,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
7,8,victim,2010-01-21,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
8,10,criminal,None,None,None,None,None,None,American Indian or Alaska Native,None,...,None,None,None,None,None,None,None,None,None,None
9,9,criminal,today,1:00pm,None,None,male,None,None,None,...,None,None,None,None,jeffrey-epstein.mp4,racksaruploads,9,https://racksaruploads.s3.amazonaws.com/9/jeff...,None,None


In [9]:
tested_id = list(pd.read_sql_query('select distinct input_prefix from match_results', open_connection())['input_prefix'])


tested_id

['11', '17']

In [23]:
sql_ = """
SELECT bucket AS input_bucket, 
       prefix AS input_prefix, 
       filename AS input_filename
FROM suspicious_activity_reports
WHERE bucket = 'racksaruploads'
AND prefix not in ({})
""".format("'" + "', '".join(tested_id) + "'")
df_suspicious = pd.read_sql_query(sql_, open_connection())
df_suspicious['hotel_image_path'] = df_suspicious['input_prefix'] +'/' + df_suspicious['input_filename']

df_suspicious['hotel_image_path'].apply(lambda row: compare_images(hotel_image_path=row, directory='victims', id_='36', file='person-wilsonzyaiaslg.jpg'))
df_suspicious

,input_bucket,input_prefix,input_filename,hotel_image_path
0,racksaruploads,11,test.jpg,11/test.jpg
1,racksaruploads,17,vallow.jpg,17/vallow.jpg


In [ ]:
compare_images(hotel_image_path='11/test.jpg', directory='victims', id_='36', file='person-wilsonzyaiaslg.jpg')

In [25]:
df_suspicious['hotel_image_path'].apply(lambda row: compare_images(hotel_image_path=row, directory='victims', id_='36', file='person-wilsonzyaiaslg.jpg'))

0    0.336352
1    0.403003
Name: hotel_image_path, dtype: float64

In [26]:
photo_compare_df = pd.read_sql_query("select * from victim_photo", open_connection())
photo_compare_df['face_vector'] = photo_compare_df['face_vector'].apply(lambda row: pickle.loads(eval(row)))
photo_compare_df = photo_compare_df.loc[photo_compare_df['face_vector'].notnull()]
photo_compare_df

,id,url,face_vector
0,36,http://www.pollyklaas.org/missing/kids/person-...,"[[144, 137, 127, 126, 125, 124, 149, 188, 182,..."
2,38,http://www.pollyklaas.org/missing/kids/images/...,"[[156, 157, 158, 158, 159, 152, 158, 164, 160,..."
3,39,http://www.pollyklaas.org/missing/kids/images/...,"[[48, 47, 44, 43, 46, 57, 71, 71, 56, 58, 76, ..."
4,40,http://www.pollyklaas.org/missing/kids/navarro...,"[[65, 35, 8, 8, 12, 9, 7, 7, 6, 5, 5, 7, 12, 1..."
5,41,http://www.pollyklaas.org/missing/kids/images/...,"[[170, 165, 159, 166, 164, 150, 158, 141, 133,..."
6,42,http://www.pollyklaas.org/missing/kids/images/...,"[[239, 237, 230, 215, 191, 169, 161, 157, 146,..."
7,43,http://www.pollyklaas.org/missing/kids/dennard...,"[[254, 253, 232, 213, 213, 212, 206, 208, 202,..."
8,44,http://www.pollyklaas.org/missing/kids/images/...,"[[210, 209, 207, 207, 201, 190, 178, 161, 145,..."
9,45,http://www.pollyklaas.org/missing/kids/images/...,"[[117, 114, 113, 133, 129, 108, 98, 111, 115, ..."
10,46,http://www.pollyklaas.org/missing/kids/images/...,"[[18, 18, 18, 19, 19, 19, 19, 19, 22, 26, 26, ..."


In [31]:
for i in photo_compare_df.index:
    id_ = photo_compare_df.loc[i, 'id']
    file = photo_compare_df.loc[i, 'url'].split('/')[-1]
    df_suspicious['similarity'] = df_suspicious['hotel_image_path'].apply(lambda row: compare_images(hotel_image_path=row, directory='victims', id_=id_, file=file))
    
    found_df = df_suspicious.loc[df_suspicious['similarity'] > .8]

ClientError: An error occurred (404) when calling the HeadObject operation: Not Found